In [1]:
import os
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from io import BytesIO
import IPython.display
import ipywidgets as widgets

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [8]:
RESOLUTION = (1280, 720)
PATH_TO_SAVED_MODEL = 'saved_model'
PATH_TO_LABELS = 'labelmap.pbtxt'
prev_id = None

model = os.path.join(os.getcwd(), 'saved_model')
labels = os.path.join(os.getcwd(), 'labelmap.pbtxt')

def showarray(a, prev_display_id=None, fmt='jpeg'):
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    obj = IPython.display.Image(data=f.getvalue())
    if prev_display_id is not None:
        IPython.display.update_display(obj, display_id=prev_display_id)
        return prev_display_id
    else:
        return IPython.display.display(obj, display_id=True)

In [3]:
#Load Model
detect_fn = tf.saved_model.load(model)
category_index = label_map_util.create_category_index_from_labelmap(labels,use_display_name=True)

In [10]:
#Test on Webcam
cap = cv2.VideoCapture(0  + cv2.CAP_DSHOW)
cap.set(3, RESOLUTION[0])
cap.set(4, RESOLUTION[1])



while(True):
    (status, frame) = cap.read()
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = frame.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=10,
          min_score_thresh=.60,
          agnostic_mode=False)

    
    prev_id = showarray(image_np_with_detections,prev_id)

ValueError: Can&#39;t clean for JSON: &lt;DisplayHandle display_id=a307a3a190e11c3dac486c3f42943b1f&gt;

In [7]:
cap.release()